<h1 style="text-align: center;">Jira Reporting Made Easy</h1>
<p style="text-align: center;"><em>Powered by Google ADK &amp; Atlassian MCP</em></p>

**This agent analyzes JIRA ticket cycle times and state workflow transitions through interactive visualizations.** It processes individual ticket changelogs to extract state transitions with timestamps, then generates Sankey diagrams, histograms, and bar charts to reveal workflow patterns and bottlenecks. 

The agent works with any JIRA instance and is perfect for product managers, engineering leads, and process improvement teams who need data-driven insights into their development workflows.

### Key Features

* Sankey Flow Diagrams - Visual workflow paths showing ticket movement between states
* State Time Histograms - Distribution analysis for cycle time spent in specific states
* Comparative Bar Charts - Total time analysis across all workflow states
* Bottleneck Detection - Automated identification of slow-moving tickets and process inefficiencies
* Data Validation - Built-in checks for impossible transitions and date inconsistencies

### Prerequisites
To run the JIRA Ticket Cycle Time & State Workflow Analytics agent with an MCP connection, you need:

* Python 3.13 with a virtual environment (venv).
* JupyterLab (installed inside the venv) or Google Colab
* Node.js - includes npx, which is used to launch the MCP client (mcp-remote) on the fly without a global install.
* Access to the Atlassian MCP endpoint: https://mcp.atlassian.com/v1/sse and permissions to read JIRA ticket data.
* JIRA instance credentials (email, API token, and base URL) configured as environment variables.

In [ ]:
!pip install google-adk mcp

In [11]:
import os
import json
import contextlib
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StdioConnectionParams, StdioServerParameters
from google.adk.models.lite_llm import LiteLlm

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
from typing import Optional
from IPython.display import display
from plotly.offline import iplot, init_notebook_mode

import aiohttp
import asyncio
import base64
import math
import re
from datetime import datetime
from google import genai

In [12]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
JIRA_EMAIL = os.getenv("JIRA_EMAIL")
JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")
JIRA_URL = os.getenv("JIRA_URL")

if GEMINI_API_KEY:
    print("Google API key loaded!")
else:
    print("Google API key missing!")

if all([JIRA_EMAIL, JIRA_API_TOKEN, JIRA_URL]):
    print("JIRA credentials loaded!")
else:
    print("JIRA credentials missing!")

Google API key loaded!
JIRA credentials loaded!


In [13]:
def clean_json_block(content: str) -> str:
    """Strip triple backticks and language identifiers from JSON content."""
    content = re.sub(r"^```json|```$", "", content.strip(), flags=re.IGNORECASE).strip()
    content = content.strip("`").strip()
    return content


async def get_jira_ticket_changelog(tool_context: ToolContext, issue_key: str) -> dict:
    """
    Gets the raw changelog for a JIRA issue key and stores it in context.
    
    Args:
        tool_context: The tool context for state management
        issue_key: JIRA issue key (e.g., "PROJ-123")
    
    Returns:
        Raw changelog dict or error dict
    """
    
    # Get config from globals (loaded in notebook)
    jira_base_url = JIRA_URL
    jira_token = JIRA_API_TOKEN
    jira_email = JIRA_EMAIL
    
    if not all([jira_base_url, jira_token, jira_email]):
        error = {"error": "JIRA configuration missing"}
        tool_context.state["current_changelog"] = error
        return error
    
    # API call
    url = f"{jira_base_url.rstrip('/')}/rest/api/2/issue/{issue_key}/changelog"
    auth = base64.b64encode(f"{jira_email}:{jira_token}".encode()).decode()
    headers = {"Authorization": f"Basic {auth}"}
    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers) as response:
                if response.status != 200:
                    error = {"error": f"API error {response.status}"}
                    tool_context.state["current_changelog"] = error
                    return error
                
                changelog = await response.json()
                
                # Store in context and return
                tool_context.state["current_changelog"] = changelog
                return changelog
                
    except Exception as e:
        error = {"error": str(e)}
        tool_context.state["current_changelog"] = error
        return error

In [14]:
async def get_jira_ticket_status_transitions(tool_context: ToolContext) -> str:
    """
    Enhanced version of get_jira_ticket_status_transitions with duplicate prevention.
    Processes the single cached JIRA changelog and extracts status transitions using LLM.
    Prevents duplicate transitions from being stored.
    
    Args:
        tool_context: The tool context for state management (contains one cached changelog)
    
    Returns:
        Success/error message as string
    """
    
    try:
        print(f"🔍 [{get_jira_ticket_status_transitions}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{get_jira_ticket_status_transitions}] Context ID: {id(tool_context.state)}")
        
        # Get the single cached changelog from tool_context
        changelog_data = tool_context.state.get("current_changelog")
        
        if not changelog_data:
            return "❌ No cached changelog found. Call get_jira_ticket_changelog first."
        
        client = genai.Client()

        # Get current date
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        system_prompt = f"""
        You are a JIRA transition data extractor. Extract status transitions from changelog data.
        
        CURRENT DATE: {current_date}
        
        CRITICAL RULES:
        1. Extract the issue key from the changelog data itself
        2. Look for actual status field changes in the changelog - entries with "field": "status"
        3. For each status change, extract the from_date, from_status, to_date, and to_status
        4. If a ticket has status transitions, create transition objects for each change
        5. If a ticket has NO status field changes, create a self-loop from creation date to current date with "Backlog" status
        6. For tickets currently in a status (last transition), create a self-loop from that status using last transition date to current date
        7. Return ONLY a JSON array of transition objects, no markdown, no extra text
        8. Each object needs: issue_key, from_date, from_status, to_date, to_status
        
        Examples:
        - With status changes: [{{"issue_key": "ABC-123", "from_date": "2024-12-13T17:01:12.419+0100", "from_status": "Backlog", "to_date": "2025-09-20T08:54:57.761+0200", "to_status": "In Progress"}}]
        - No status changes: [{{"issue_key": "ABC-123", "from_date": "2024-12-13T17:01:12.419+0100", "from_status": "Backlog", "to_date": "{current_date}T00:00:00.000+0000", "to_status": "Backlog"}}]
        """
        
        # Pass the single changelog to LLM
        user_prompt = f"""
        Extract status transitions from this JIRA changelog data:
        
        {json.dumps(changelog_data, indent=2)}
        
        Remember: 
        - Look for "field": "status" entries for actual status changes
        - Extract the exact dates from the "created" field of each entry
        - If no status changes found, use creation date and current date with "Backlog" status
        - DO NOT calculate days - only extract dates and statuses
        
        Return only the JSON array of transition objects following the exact schema.
        """
        
        # Make Gemini API call
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[
                {"role": "user", "parts": [{"text": system_prompt + "\n\n" + user_prompt}]}
            ]
        )
        
        content = response.text.strip()
        print("Raw Gemini response:\n", content)
        
        # Clean and parse JSON
        import re
        content = re.sub(r"^```json|```$", "", content.strip(), flags=re.IGNORECASE).strip()
        content = content.strip("`").strip()
        transitions = json.loads(content)
        
        # Validate response
        if not isinstance(transitions, list):
            return f"❌ Expected list of transitions, got: {type(transitions)}"
        
        # Extract creation date from changelog for validation
        creation_date = None
        ticket_lifetime_days = None
        
        # Look for the earliest date in the changelog as creation date
        if "values" in changelog_data:
            for history in changelog_data["values"]:
                if "created" in history:
                    creation_date = history["created"]
                    break
        
        # Calculate ticket lifetime (creation to now) for validation
        if creation_date:
            try:
                if "T" in creation_date:
                    created = datetime.fromisoformat(creation_date.replace("Z", "+00:00"))
                else:
                    created = datetime.strptime(creation_date, "%Y-%m-%d")
                
                current = datetime.now(created.tzinfo) if created.tzinfo else datetime.now()
                ticket_lifetime_days = (current - created).days + 1
                print(f"Ticket lifetime: {ticket_lifetime_days} days (created: {created.strftime('%Y-%m-%d')})")
            except Exception as e:
                print(f"⚠️ Could not parse creation date: {e}")
                ticket_lifetime_days = None
        
        # Initialize storage if needed
        if "ticket_status_transitions" not in tool_context.state:
            tool_context.state["ticket_status_transitions"] = []
        
        # Handle empty transitions - now this should be rare since we want self-loops
        if not transitions:
            print("⚠️ Warning: No transitions returned - this is unexpected")
            return "✅ Processed: +0 new, -0 duplicates, no transitions returned"
        
        # Get existing transitions for duplicate checking
        existing_transitions = tool_context.state["ticket_status_transitions"]
        existing_keys = set()
        for existing in existing_transitions:
            key = (existing["issue_key"], existing["source_status"], existing["target_status"])
            existing_keys.add(key)
        
        # Add transitions to storage with duplicate prevention
        valid_count = 0
        duplicate_count = 0
        extracted_transitions = []
        
        for transition in transitions:
            required_keys = ["issue_key", "from_date", "from_status", "to_date", "to_status"]
            if all(key in transition for key in required_keys):
                
                # Calculate transition_days using the tool, not the model
                try:
                    from_date_str = transition["from_date"]
                    to_date_str = transition["to_date"]
                    
                    # Parse dates
                    if "T" in from_date_str:
                        from_date = datetime.fromisoformat(from_date_str.replace("Z", "+00:00"))
                    else:
                        from_date = datetime.strptime(from_date_str, "%Y-%m-%d")
                    
                    if "T" in to_date_str:
                        to_date = datetime.fromisoformat(to_date_str.replace("Z", "+00:00"))
                    else:
                        to_date = datetime.strptime(to_date_str, "%Y-%m-%d")
                    
                    # Calculate days (minimum 1)
                    days = max(1, (to_date - from_date).days + 1)
                    
                    # Create final transition object with calculated days
                    final_transition = {
                        "issue_key": transition["issue_key"],
                        "source_status": transition["from_status"],
                        "target_status": transition["to_status"],
                        "transition_days": days
                    }
                    
                except Exception as e:
                    print(f"❌ Date parsing error for {transition['issue_key']}: {e}")
                    continue
                
                # Check for duplicates using final transition format
                key = (final_transition["issue_key"], final_transition["source_status"], final_transition["target_status"])
                
                if key in existing_keys:
                    duplicate_count += 1
                    print(f"🔄 Skipped duplicate: {final_transition['issue_key']} {final_transition['source_status']} → {final_transition['target_status']}")
                else:
                    # Check if transition days exceed ticket lifetime (impossible scenario)
                    if ticket_lifetime_days and final_transition["transition_days"] > ticket_lifetime_days:
                        print(f"❌ INVALID DATA: {final_transition['issue_key']} shows {final_transition['transition_days']} days but ticket only exists for {ticket_lifetime_days} days. Skipping this transition.")
                        continue  # Skip this invalid transition
                    
                    tool_context.state["ticket_status_transitions"].append(final_transition)
                    existing_keys.add(key)  # Add to tracking set
                    extracted_transitions.append(final_transition)
                    valid_count += 1
        
        # Print results for visibility in notebook
        print(f"\n{'='*60}")
        print(f"STATUS TRANSITIONS PROCESSED")
        print(f"{'='*60}")
        print(f"✅ Added: {valid_count} new transitions")
        if duplicate_count > 0:
            print(f"🔄 Skipped: {duplicate_count} duplicates")
        print(f"📊 Total in context: {len(tool_context.state['ticket_status_transitions'])} transitions")
        
        if extracted_transitions:
            print(f"\n📈 New transitions added:")
            for i, transition in enumerate(extracted_transitions, 1):
                print(f"{i}. {transition['issue_key']}: {transition['source_status']} → {transition['target_status']} ({transition['transition_days']} days)")
        
        print(f"{'='*60}\n")
        
        return f"✅ Processed: +{valid_count} new, -{duplicate_count} duplicates, {len(tool_context.state['ticket_status_transitions'])} total"
        
    except json.JSONDecodeError as e:
        return f"❌ Failed to parse Gemini response: {str(e)}\nResponse: {content}"
    except Exception as e:
        return f"❌ Error processing changelog: {str(e)}"

In [15]:
def create_sankey_diagram(tool_context: ToolContext, title: str = "JIRA Status Flow") -> str:
    """
    Creates a Sankey diagram from transition data stored in tool_context.
    
    Args:
        tool_context: The tool context containing ticket_status_transitions data
        title: Title for the diagram
    
    Returns:
        Success/error message as string
        
    Expected data in tool_context.state["ticket_status_transitions"]:
    [
        {
            "issue_key": "PROJ-123",
            "source_status": "Backlog", 
            "target_status": "In Progress",
            "transition_days": 3
        }
    ]
    """
    
    try:
        print(f"🔍 [{create_sankey_diagram}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{create_sankey_diagram}] Context ID: {id(tool_context.state)}")
        
        # Initialize notebook mode for plotly
        init_notebook_mode(connected=True)
        
        # Get transition data from context
        transitions = tool_context.state.get("ticket_status_transitions", [])
        
        if not transitions:
            return "❌ No transition data found in context. Run get_jira_ticket_status_transitions first."
        
        # Create and display DataFrame of raw transition data
        df = pd.DataFrame(transitions)
        df['note'] = df.apply(lambda row: 
            "(self-loop)" if row['source_status'] == row['target_status']
            else "(from beginning)" if row['source_status'] in {"New", "Backlog"}
            else "(to end)" if row['target_status'] in {"Done", "Closed"}
            else "", axis=1)
        
        print(f"\n📋 Raw Transition Data for Sankey Diagram")
        print(f"{'='*80}")
        display(df[['issue_key', 'source_status', 'target_status', 'transition_days', 'note']])
        print(f"{'='*80}\n")
        
        # Define end statuses that should not have "Currently in X" nodes
        END_STATUSES = {"Done", "Closed"}
        
        # Aggregate transitions and collect ticket details
        aggregated = {}
        all_statuses = set()
        
        for transition in transitions:
            source = transition["source_status"]
            target = transition["target_status"]
            days = transition["transition_days"]
            ticket = transition["issue_key"]
            
            all_statuses.add(source)
            all_statuses.add(target)
            
            key = (source, target)
            if key not in aggregated:
                aggregated[key] = {"total_days": 0, "tickets": []}
            
            aggregated[key]["total_days"] += days
            aggregated[key]["tickets"].append(f"{ticket}:{days}d")

        # Handling of self-loops - end statuses are excluded
        new_aggregated = {}
        for (source, target), data in aggregated.items():
            if source == target:
                # Only create virtual nodes for non-end statuses
                if target not in END_STATUSES:
                    # Create virtual node: "Currently in {status}"
                    virtual_status = f"Currently in {source}"
                    all_statuses.add(virtual_status)
                    # Use virtual status as target
                    new_aggregated[(source, virtual_status)] = data
                # For end statuses, skip creating the self-loop entirely
                # This means tickets that end in Done/Closed won't show artificial flows
            else:
                new_aggregated[(source, target)] = data
        
        aggregated = new_aggregated
        
        if not aggregated:
            return "❌ No valid transitions found to visualize."
        
        # Create status mapping
        # status_list = sorted(list(all_statuses))
        # status_to_id = {status: idx for idx, status in enumerate(status_list)}

        #START fix sorting
        # Create status mapping with end statuses positioned last
        end_statuses_in_data = [s for s in all_statuses if s in END_STATUSES]
        non_end_statuses = [s for s in all_statuses if s not in END_STATUSES]
        
        # Sort each group separately, then combine with end statuses last
        status_list = sorted(non_end_statuses) + sorted(end_statuses_in_data)
        status_to_id = {status: idx for idx, status in enumerate(status_list)}
        #END fix sorting
        
        # Build nodes (let Plotly handle colors)
        node_labels = status_list
        
        # Build links with ticket information
        link_sources = []
        link_targets = []
        link_values = []
        link_labels = []
        
        for (source, target), data in aggregated.items():
            source_id = status_to_id[source]
            target_id = status_to_id[target]
            total_days = data["total_days"]
            tickets = data["tickets"]
            
            link_sources.append(source_id)
            link_targets.append(target_id)
            link_values.append(total_days)
            
            # Create hover label with ticket details
            if source == target:
                flow_text = f"{source} (current)"
            else:
                flow_text = f"{source} → {target}"
            
            ticket_list = ", ".join(tickets)
            hover_label = f"{flow_text}: {total_days} days<br>Tickets: {ticket_list}"
            link_labels.append(hover_label)
        
        # Create Sankey diagram
        fig = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=node_labels
                # Let Plotly auto-assign colors
            ),
            link=dict(
                source=link_sources,
                target=link_targets,
                value=link_values,
                label=link_labels,
                hovertemplate='%{label}<extra></extra>'
                # Let Plotly auto-assign colors
            )
        )])
        
        # Update layout
        fig.update_layout(
            title_text=f"{title}<br><sub>Flow values represent total days spent in transitions</sub>",
            font_size=12,
            width=1000,
            height=600
        )
        
        # Display the diagram
        iplot(fig)
        
        # Print summary
        total_transitions = len(transitions)
        unique_tickets = len(set(t["issue_key"] for t in transitions))
        total_days = sum(t["transition_days"] for t in transitions)
        
        print(f"\n📊 Sankey Diagram Created: {title}")
        print(f"   • {total_transitions} transitions visualized")
        print(f"   • {unique_tickets} unique tickets")
        print(f"   • {total_days} total days tracked")
        print(f"   • Hover over flows to see ticket details\n")
        
        return f"✅ Sankey diagram created with {total_transitions} transitions"
        
    except Exception as e:
        return f"❌ Error creating Sankey diagram: {str(e)}"

In [16]:
def create_jira_ticket_status_time_bar_chart(tool_context: ToolContext, title: str = "Time Spent in Each Status") -> str:
    """
    Creates a bar chart showing total time spent in each status across all tickets. 
    Historical completed time in each status and not the current/ongoing time for tickets in the given status.
    Excludes beginning statuses (New, Backlog) and end statuses (Done, Closed).
    
    Args:
        tool_context: The tool context containing ticket_status_transitions data
        title: Title for the chart
    
    Returns:
        Success/error message as string
        
    Expected data in tool_context.state["ticket_status_transitions"]:
    [
        {
            "issue_key": "PROJ-123",
            "source_status": "Backlog", 
            "target_status": "In Progress",
            "transition_days": 3
        }
    ]
    """
    
    try:
        print(f"🔍 [{create_jira_ticket_status_time_bar_chart}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{create_jira_ticket_status_time_bar_chart}] Context ID: {id(tool_context.state)}")
        
        # Initialize notebook mode for plotly
        init_notebook_mode(connected=True)
        
        # Get transition data from context
        transitions = tool_context.state.get("ticket_status_transitions", [])
        
        if not transitions:
            return "❌ No transition data found in context. Run get_jira_ticket_status_transitions first."
        
        # Display raw transition data as DataFrame (same as other tools)
        df = pd.DataFrame(transitions)
        df['note'] = df.apply(lambda row: 
            "(self-loop)" if row['source_status'] == row['target_status']
            else "(from beginning)" if row['source_status'] in {"New", "Backlog"}
            else "(to end)" if row['target_status'] in {"Done", "Closed"}
            else "", axis=1)
        
        print(f"\n📋 Raw Transition Data for Bar Chart Analysis")
        print(f"{'='*80}")
        display(df[['issue_key', 'source_status', 'target_status', 'transition_days', 'note']])
        print(f"{'='*80}\n")
        
        # Define statuses to exclude from the chart
        BEGINNING_STATUSES = {"New", "Backlog"}
        END_STATUSES = {"Done", "Closed"}
        EXCLUDED_STATUSES = BEGINNING_STATUSES | END_STATUSES
        
        # Aggregate time spent in each status (INCLUDE SELF-LOOPS)
        status_time = defaultdict(int)
        status_tickets = defaultdict(list)  # Track which tickets contributed to each status
        
        for transition in transitions:
            source_status = transition["source_status"]
            target_status = transition["target_status"]
            days = transition["transition_days"]
            ticket = transition["issue_key"]
            
            # Time spent IN a status includes both transitions FROM that status AND self-loops
            if source_status not in EXCLUDED_STATUSES:
                status_time[source_status] += days
                if source_status == target_status:
                    status_tickets[source_status].append(f"{ticket}:{days}d (self-loop)")
                else:
                    status_tickets[source_status].append(f"{ticket}:{days}d")
        
        if not status_time:
            return "❌ No valid status time data found. All transitions involve beginning/end statuses."
        
        print(f"🔍 Time aggregation for each status:")
        for status in sorted(status_time.keys()):
            total_time = status_time[status]
            tickets = status_tickets[status]
            print(f"   • {status}: {total_time} days ({len(tickets)} entries)")
            for ticket in tickets:
                print(f"     - {ticket}")
        print()
        
        # Sort statuses by total time (descending)
        sorted_statuses = sorted(status_time.items(), key=lambda x: x[1], reverse=True)
        
        statuses = [status for status, _ in sorted_statuses]
        times = [time for _, time in sorted_statuses]
        
        # Create hover text with ticket details
        hover_texts = []
        for status in statuses:
            total_time = status_time[status]
            tickets = status_tickets[status]
            ticket_count = len(tickets)
            ticket_list = ", ".join(tickets[:5])  # Show first 5 tickets
            if len(tickets) > 5:
                ticket_list += f" (+{len(tickets)-5} more)"
            
            hover_text = f"Status: {status}<br>Total Days: {total_time}<br>Entries: {ticket_count}<br>Details: {ticket_list}"
            hover_texts.append(hover_text)
        
        # Create bar chart
        fig = go.Figure(data=[
            go.Bar(
                x=statuses,
                y=times,
                text=[f"{time}d" for time in times],  # Show days on bars
                textposition='auto',
                hovertemplate='%{customdata}<extra></extra>',
                customdata=hover_texts,
                marker=dict(
                    color=times,
                    colorscale='Viridis',
                    showscale=False,
                    colorbar=dict(title="Days")
                )
            )
        ])
        
        # Update layout
        fig.update_layout(
            title=f"{title}<br><sub>Includes self-loops, excludes beginning (New, Backlog) and end (Done, Closed) statuses</sub>",
            xaxis_title="Status",
            yaxis_title="Total Days",
            font_size=12,
            width=1000,
            height=600,
            xaxis={'categoryorder': 'total descending'},  # Ensure bars are sorted by value
            yaxis=dict(
                dtick=1,  # Set tick interval to 1 day
                tick0=0   # Start ticks at 0
            )
        )
        
        # Display the chart
        iplot(fig)
        
        # Print summary
        total_statuses = len(statuses)
        total_days = sum(times)
        unique_tickets = len(set(t["issue_key"] for t in transitions))
        
        print(f"\n📊 Status Time Bar Chart Created: {title}")
        print(f"   • {total_statuses} active statuses analyzed")
        print(f"   • {total_days} total days across all statuses (includes self-loops)")
        print(f"   • {unique_tickets} unique tickets")
        print(f"   • Excludes: {', '.join(sorted(EXCLUDED_STATUSES))}")
        print(f"   • Hover over bars to see ticket details\n")
        
        # Print top statuses
        print("📈 Top statuses by time spent:")
        for i, (status, time) in enumerate(sorted_statuses[:5], 1):
            entry_count = len(status_tickets[status])
            print(f"   {i}. {status}: {time} days ({entry_count} entries)")
        
        if len(sorted_statuses) > 5:
            print(f"   ... and {len(sorted_statuses) - 5} more statuses")
        print()
        
        return f"✅ Bar chart created showing {total_statuses} statuses with {total_days} total days (includes self-loops)"
        
    except Exception as e:
        return f"❌ Error creating status time bar chart: {str(e)}"

In [17]:
def create_jira_ticket_status_histogram(tool_context: ToolContext, status: str, title: Optional[str] = None) -> str:
    """
    Creates a histogram showing the distribution of time spent in a specific status.
    Shows time intervals from 1-30 days (individual buckets) plus 30+ days bucket.
    Includes statistical annotations for bottleneck analysis.
    
    Args:
        tool_context: The tool context containing ticket_status_transitions data
        status: The specific status to analyze (case-sensitive)
        title: Optional custom title for the chart
    
    Returns:
        Success/error message as string
        
    Expected data in tool_context.state["ticket_status_transitions"]:
    [
        {
            "issue_key": "PROJ-123",
            "source_status": "In Progress", 
            "target_status": "Code Review",
            "transition_days": 3
        }
    ]
    """
    
    try:
        print(f"🔍 [{create_jira_ticket_status_histogram}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{create_jira_ticket_status_histogram}] Context ID: {id(tool_context.state)}")
        
        # Initialize notebook mode for plotly
        init_notebook_mode(connected=True)
        
        # Get transition data from context
        transitions = tool_context.state.get("ticket_status_transitions", [])
        
        if not transitions:
            return "❌ No transition data found in context. Run get_jira_ticket_status_transitions first."
        
        # Display raw transition data as DataFrame (same as Sankey)
        df = pd.DataFrame(transitions)
        df['note'] = df.apply(lambda row: 
            "(self-loop)" if row['source_status'] == row['target_status']
            else "(from beginning)" if row['source_status'] in {"New", "Backlog"}
            else "(to end)" if row['target_status'] in {"Done", "Closed"}
            else "", axis=1)
        
        print(f"\n📋 Raw Transition Data for Histogram Analysis")
        print(f"{'='*80}")
        display(df[['issue_key', 'source_status', 'target_status', 'transition_days', 'note']])
        print(f"{'='*80}\n")
        
        # Define statuses to exclude from analysis
        BEGINNING_STATUSES = {"New", "Backlog"}
        END_STATUSES = {"Done", "Closed"}
        EXCLUDED_STATUSES = BEGINNING_STATUSES | END_STATUSES
        
        # Validate the requested status
        if status in EXCLUDED_STATUSES:
            return f"❌ Cannot analyze '{status}' - it's a beginning or end status. Choose from active workflow statuses."
        
        # Collect time data for the specified status (INCLUDE SELF-LOOPS)
        # status_times = []
        # ticket_details = []  # For hover information
        
        # for transition in transitions:
        #     source_status = transition["source_status"]
        #     target_status = transition["target_status"]
        #     days = transition["transition_days"]
        #     ticket = transition["issue_key"]
            
        #     # We want time spent IN the specified status
        #     # This includes both transitions FROM that status AND self-loops IN that status
        #     if source_status == status:
        #         status_times.append(days)
        #         if source_status == target_status:
        #             ticket_details.append(f"{ticket} ({days}d, self-loop)")
        #         else:
        #             ticket_details.append(f"{ticket} ({days}d)")

        # Collect time data for the specified status (SUM TIME PER TICKET)
        ticket_time_map = {}  # ticket_key -> total_days
        ticket_details = []   # For hover information
        
        for transition in transitions:
            source_status = transition["source_status"]
            target_status = transition["target_status"]
            days = transition["transition_days"]
            ticket = transition["issue_key"]
            
            # We want time spent IN the specified status
            # This includes both transitions FROM that status AND self-loops IN that status
            if source_status == status:
                if ticket not in ticket_time_map:
                    ticket_time_map[ticket] = 0
                ticket_time_map[ticket] += days
        
        # Convert to lists for further processing
        status_times = list(ticket_time_map.values())
        ticket_details = [f"{ticket} ({total_days}d)" for ticket, total_days in ticket_time_map.items()]
        
        if not status_times:
            available_statuses = set()
            for t in transitions:
                if t["source_status"] not in EXCLUDED_STATUSES:
                    available_statuses.add(t["source_status"])
            
            available_list = sorted(list(available_statuses))
            return f"❌ No time data found for status '{status}'. Available statuses: {available_list}"
        
        print(f"🔍 Found {len(status_times)} time entries for '{status}' status:")
        for detail in ticket_details:
            print(f"   • {detail}")
        print()
        
        # Calculate statistics
        times_array = np.array(status_times)
        stats = {
            'count': len(status_times),
            'median': np.median(times_array),
            'mean': np.mean(times_array),
            'p90': np.percentile(times_array, 90),
            'min': np.min(times_array),
            'max': np.max(times_array)
        }
        
        # Create histogram buckets: 1-30 days individually, then 30+ days
        max_individual_bucket = 30
        
        # Create bins: [0.5, 1.5, 2.5, ..., 30.5, inf]
        bins = [i + 0.5 for i in range(max_individual_bucket + 1)]
        bins.append(float('inf'))
        
        # Calculate histogram
        hist_counts, _ = np.histogram(status_times, bins=bins)
        
        # Only show bins that have data or are within reasonable range of data
        max_data_day = min(int(np.max(times_array)), max_individual_bucket)
        display_range = max(max_data_day + 2, 5)  # Show at least 5 days, or max + 2
        display_range = min(display_range, max_individual_bucket)
        
        # Create bin labels and data for display range only
        bin_labels = []
        bin_counts = []
        hover_texts = []
        
        for i in range(display_range):
            day = i + 1
            bin_labels.append(f"{day}d")
            bin_counts.append(hist_counts[i])
            
            # Create hover text
            tickets_in_bin = [detail for detail, time in zip(ticket_details, status_times) if time == day]
            
            if hist_counts[i] > 0:
                ticket_list = ", ".join(tickets_in_bin[:5])  # Show first 5 tickets
                if len(tickets_in_bin) > 5:
                    ticket_list += f" (+{len(tickets_in_bin)-5} more)"
                hover_text = f"Time: {day}d<br>Tickets: {hist_counts[i]}<br>Examples: {ticket_list}"
            else:
                hover_text = f"Time: {day}d<br>Tickets: 0"
            
            hover_texts.append(hover_text)
        
        # Add 30+ bucket if there's data beyond display range
        if np.max(times_array) > max_individual_bucket:
            bin_labels.append("30+d")
            bin_counts.append(hist_counts[-1])  # Last bin is 30+ days
            
            tickets_in_bin = [detail for detail, time in zip(ticket_details, status_times) if time > max_individual_bucket]
            
            if hist_counts[-1] > 0:
                ticket_list = ", ".join(tickets_in_bin[:5])
                if len(tickets_in_bin) > 5:
                    ticket_list += f" (+{len(tickets_in_bin)-5} more)"
                hover_text = f"Time: 30+d<br>Tickets: {hist_counts[-1]}<br>Examples: {ticket_list}"
            else:
                hover_text = f"Time: 30+d<br>Tickets: 0"
            
            hover_texts.append(hover_text)
        
        # Create the histogram
        fig = go.Figure(data=[
            go.Bar(
                x=bin_labels,
                y=bin_counts,
                text=bin_counts,
                textposition='auto',
                hovertemplate='%{customdata}<extra></extra>',
                customdata=hover_texts,
                marker=dict(
                    color=bin_counts,
                    colorscale='Blues',
                    showscale=False,
                    colorbar=dict(title="Ticket Count")
                )
            )
        ])
        
        # Add statistical annotation lines (only if they're at different positions)
        if abs(stats['median'] - stats['p90']) > 0.1:
            # Different positions - show both lines
            fig.add_vline(
                x=stats['median'] - 0.5, 
                line_dash="dash", 
                line_color="red", 
                annotation_text=f"Median: {stats['median']:.1f}d",
                annotation_position="top left"
            )
            
            fig.add_vline(
                x=stats['p90'] - 0.5, 
                line_dash="dot", 
                line_color="orange", 
                annotation_text=f"90th percentile: {stats['p90']:.1f}d",
                annotation_position="top right"
            )
        else:
            # Same or very close positions - show combined line
            fig.add_vline(
                x=stats['median'] - 0.5, 
                line_dash="dash", 
                line_color="red", 
                annotation_text=f"Median & 90th percentile: {stats['median']:.1f}d",
                annotation_position="top"
            )
        
        # Set title
        if title is None:
            title = f"Time Distribution for Status: {status}"
        
        # Update layout with improved x-axis (FIX THE GAP ISSUE)
        fig.update_layout(
            title=f"{title}<br><sub>Distribution of days spent in '{status}' status (includes self-loops)</sub>",
            xaxis_title="Days Spent in Status",
            yaxis_title="Number of Tickets",
            font_size=12,
            width=1200,
            height=600,
            showlegend=False,
            xaxis=dict(
                type='category',
                categoryorder='array',
                categoryarray=bin_labels,
                # Remove gaps between bars
                range=[-0.5, len(bin_labels) - 0.5]
            ),
            # Remove default margins that cause gaps
            margin=dict(l=50, r=50, t=80, b=50)
        )
        
        # Display the chart
        iplot(fig)
        
        # Print detailed statistics
        print(f"\n📊 Status Time Distribution: {status}")
        print(f"{'='*50}")
        print(f"📈 Statistics:")
        print(f"   • Total tickets analyzed: {stats['count']} (includes self-loops)")
        print(f"   • Average time: {stats['mean']:.1f} days")
        print(f"   • Median time: {stats['median']:.1f} days")
        print(f"   • 90th percentile: {stats['p90']:.1f} days")
        print(f"   • Range: {stats['min']:.0f} - {stats['max']:.0f} days")
        
        # Bottleneck analysis
        print(f"\n🔍 Bottleneck Analysis:")
        quick_tickets = sum(1 for t in status_times if t <= 3)
        slow_tickets = sum(1 for t in status_times if t > 7)
        very_slow_tickets = sum(1 for t in status_times if t > 14)
        
        quick_pct = (quick_tickets / stats['count']) * 100
        slow_pct = (slow_tickets / stats['count']) * 100
        very_slow_pct = (very_slow_tickets / stats['count']) * 100
        
        print(f"   • Fast (≤3 days): {quick_tickets} tickets ({quick_pct:.1f}%)")
        print(f"   • Slow (>7 days): {slow_tickets} tickets ({slow_pct:.1f}%)")
        print(f"   • Very slow (>14 days): {very_slow_tickets} tickets ({very_slow_pct:.1f}%)")
        
        # Recommendations
        print(f"\n💡 Insights:")
        if stats['p90'] > stats['median'] * 3:
            print(f"   ⚠️  High variability detected - some tickets take much longer")
        if very_slow_pct > 10:
            print(f"   🚨 Bottleneck warning - {very_slow_pct:.1f}% of tickets are very slow")
        if quick_pct > 80:
            print(f"   ✅ Good flow - most tickets move quickly through {status}")
        
        print(f"{'='*50}\n")
        
        return f"✅ Histogram created for '{status}' status with {stats['count']} tickets analyzed (includes self-loops)"
        
    except Exception as e:
        return f"❌ Error creating status histogram: {str(e)}"

In [18]:
def clear_status_transitions_data_set(tool_context: ToolContext) -> str:
    """
    Clears all cached transition data to provide a clean slate for new analysis.
    
    Args:
        tool_context: The tool context for state management
    
    Returns:
        Success message as string
    """
    
    try:
        print(f"🔍 [{clear_status_transitions_data_set}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{clear_status_transitions_data_set}] Context ID: {id(tool_context.state)}")
        
        # Clear all relevant data
        keys_to_clear = ["ticket_status_transitions", "current_changelog"]
        cleared_keys = []
        
        for key in keys_to_clear:
            if key in tool_context.state:
                del tool_context.state[key]
                cleared_keys.append(key)
        
        if cleared_keys:
            print("Data cleared successfully!")
            print(f"   • Cleared: {', '.join(cleared_keys)}")
            print("   • Ready for fresh analysis")
            return f"Status transitions set is now empty - cleared {len(cleared_keys)} data stores: {', '.join(cleared_keys)}"
        else:
            print("Status transitions set is empty!")
            return "Status transitions set is empty - context is clean"
        
    except Exception as e:
        print(f"❌ Clear error: {str(e)}")
        return f"Error clearing data: {str(e)}"


def get_status_transitions_summary(tool_context: ToolContext) -> str:
    """
    Shows a summary of currently stored transition data without creating visualizations.
    Displays data in a nice DataFrame table for easy analysis.
    
    Args:
        tool_context: The tool context for state management
    
    Returns:
        Summary information as string
    """
    
    try:
        print(f"🔍 [{get_status_transitions_summary}] State check: {len(tool_context.state.get('ticket_status_transitions', []))} transitions in context")
        print(f"🔍 [{get_status_transitions_summary}] Context ID: {id(tool_context.state)}")
        
        transitions = tool_context.state.get("ticket_status_transitions", [])
        current_changelog = tool_context.state.get("current_changelog")
        
        print("Current Status Transitions Data Summary")
        print("=" * 50)
        
        if transitions:
            unique_tickets = set(t["issue_key"] for t in transitions)
            unique_statuses = set()
            for t in transitions:
                unique_statuses.add(t["source_status"])
                unique_statuses.add(t["target_status"])
            
            total_days = sum(t["transition_days"] for t in transitions)
            
            print("Transition Data:")
            print(f"   • {len(transitions)} total transitions")
            print(f"   • {len(unique_tickets)} unique tickets: {', '.join(sorted(unique_tickets))}")
            print(f"   • {len(unique_statuses)} statuses involved: {', '.join(sorted(unique_statuses))}")
            print(f"   • {total_days} total days tracked")
            
            # Create and display DataFrame from transitions data
            df = pd.DataFrame(transitions)
            print("\nAll Transition Data:")
            display(df)
                
        else:
            print("Transition Data: None - status transitions set is empty")
        
        if current_changelog:
            if "error" in current_changelog:
                print(f"Cached Changelog: Error - {current_changelog['error']}")
            else:
                issue_key = current_changelog.get("self", "unknown")
                if "/browse/" in issue_key:
                    issue_key = issue_key.split("/browse/")[-1]
                print(f"Cached Changelog: Available for {issue_key}")
        else:
            print("Cached Changelog: None")
        
        print("=" * 50)
        
        if transitions:
            return f"Summary: {len(transitions)} transitions from {len(unique_tickets)} tickets ready for analysis"
        else:
            return "Status transitions set is empty - run get_jira_ticket_changelog and get_jira_ticket_status_transitions first"
        
    except Exception as e:
        return f"Error generating summary: {str(e)}"

In [19]:
agent = Agent(
    name="product_doc_assistant",
    model="gemini-2.5-flash",
    description="Jira assistant with auto-discovered Atlassian MCP tools for jenys.atlassian.net",
    instruction="""
            You are a specialized AI assistant designed to help users analyze JIRA tickets and create status flow visualizations. You are working
             in the jenys.atlassian.net domain.
             
            **Your Goal**:
            - Help users analyze JIRA ticket workflows and create Sankey diagrams showing status transitions. 
            - Suggest also other type of diagrams for extended data analysis - bar chart or histogram.
            
            **Your Workflow**:
            Always follow these steps precisely!
            
            1. **Analyze the user's question** to determine their specific intent.
            
            2. **For JIRA ticket analysis requests:**
               - Use `get_jira_ticket_changelog` tool to retrieve changelog data for each ticket
               - Use `get_jira_ticket_status_transitions` tool to process the changelog and extract status transitions
               - Repeat steps above for each ticket the user wants to analyze
               - Use `create_sankey_diagram` tool to create a visualization of all collected transitions. Only create diagrams after collecting all ticket transitions!
            
            3. **For status transition analysis:**
               - Each ticket should be processed individually: get changelog → extract transitions
               - Transitions are automatically accumulated in context for final visualization
               - If a ticket has never moved from its initial status, record it as a self-loop (source and target are the same)
               - Use `get_status_transitions_summary` to show what data is available
            
            4. **For starting fresh analysis on a new data set:**
               - Use `clear_status_transitions_data_set` if user wants to start **fresh analysis**.
               - Always call `get_status_transitions_summary` after clearing to confirm the data has been cleared.

            5. **For getting an overview of the extracted tickets and their status transitions:**
               - Use `get_status_transitions_summary` to show what data is available. If there is no data extracted yet, then inform the user that their data set is currently empty.
            
            **Crucial Rules**:
            - **Handling Out-of-Scope Requests:** If a user's question is not related to JIRA ticket analysis or status flow visualization, politely decline and explain your capabilities are limited to JIRA workflow analysis.
            - **Data Processing:** Always process tickets one at a time - get changelog, then extract transitions, before moving to the next ticket.
            - **Accurate Analysis:** If you cannot find changelog data or encounter errors, explain what went wrong and suggest alternative approaches.
            
            **Hard Rule**:
            - **Follow the workflow sequence**: Always get changelog first, then extract transitions, then visualize. Do not skip steps.
            
            Be helpful. Be accurate. Be effective.
            """,
    tools=[
        get_jira_ticket_changelog,
        get_jira_ticket_status_transitions,
        create_sankey_diagram,
        create_jira_ticket_status_time_bar_chart,
        create_jira_ticket_status_histogram,
        clear_status_transitions_data_set,
        get_status_transitions_summary,
        
        MCPToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        '-y',  # Auto-confirm npx install
                        'mcp-remote',
                        'https://mcp.atlassian.com/v1/sse'
                    ],
                ),
                timeout=300 
            ),
            # tool_filter=['getConfluenceSpaces', 'getConfluencePage'] # Example how to filter tools - only the listed ones are then allowed.
        ),
    ],
)


# Setup session and runner  
session_service = InMemorySessionService()
runner = Runner(agent=agent, app_name="product_doc_assistant", session_service=session_service)

# Global session setup
USER_ID = "user"
SESSION_ID = "product_doc_conversation"
session_initialized = False


async def chat():
    """Start interactive conversation with the agent"""
    global session_initialized
    
    if not session_initialized:
        await session_service.create_session(
            app_name="product_doc_assistant",
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        session_initialized = True
        print("🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit', 'exit', 'stop' to end our chat.")
    
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("👋 Goodbye!")
            break
            
        if not user_input:
            continue
            
        message = types.Content(role='user', parts=[types.Part(text=user_input)])
        print("Agent: ", end="", flush=True)
        
        with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f):
            async for event in runner.run_async(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=message
            ):
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if part.text:
                            print(part.text, end="", flush=True)
        
        print("\n")

# [DEBUG] Use this to print the full execution output
# async def chat():
#     """Start interactive conversation with the agent"""
#     global session_initialized
    
#     if not session_initialized:
#         await session_service.create_session(
#             app_name="product_doc_assistant",
#             user_id=USER_ID,
#             session_id=SESSION_ID
#         )
#         session_initialized = True
#         print("🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit' to end our chat.")
    
#     while True:
#         user_input = input("You: ").strip()
        
#         if user_input.lower() in ['quit', 'exit', 'stop']:
#             print("👋 Goodbye!")
#             break
            
#         if not user_input:
#             continue
            
#         message = types.Content(role='user', parts=[types.Part(text=user_input)])
#         print("Agent: ", end="", flush=True)
        
#         events = []
#         async for event in runner.run_async(
#             user_id=USER_ID,
#             session_id=SESSION_ID,
#             new_message=message
#         ):
#             events.append(event)
#             print("\n────────── RAW EVENT ──────────")
#             print(json.dumps(event.model_dump(), indent=2, default=str))
#             print(
#                 f"[{type(event).__name__:<25}] "
#                 f"author={event.author:<15} "
#                 f"final={event.is_final_response()}"
#             )
            
#             # Print content parts (LLM output)
#             if event.content and event.content.parts:
#                 for part in event.content.parts:
#                     if part.text:
#                         print("💬 text →", part.text.strip())
            
#             # Tool calls
#             for call in event.get_function_calls():
#                 print(f"🔧 tool‐call → {call.name}({json.dumps(call.args, indent=2)})")
            
#             # Tool results
#             for resp in event.get_function_responses():
#                 print(f"[TOOL] tool‐result → {resp.name} → {json.dumps(resp.response.model_dump() if hasattr(resp.response, 'model_dump') else str(resp.response), indent=2, default=str)}")
            
#             # State delta
#             if event.actions and event.actions.state_delta:
#                 print("[STATE] state Δ →", json.dumps(event.actions.state_delta, indent=2))
#             if event.actions and event.actions.artifact_delta:
#                 print("[ARTIFACT] artifact Δ →", json.dumps(event.actions.artifact_delta, indent=2))
        
#         # Final output
#         final_text = (
#             events[-1].content.parts[0].text
#             if events and events[-1].content and events[-1].content.parts else ""
#         )
#         print("\n===== ✅ FINAL ANSWER =====")
#         print(final_text or "(no text)")
#         print("\n")

In [20]:
await chat()

🤖 Hello! I'm here to assist with your documentation needs. Please ask me a question about the product, or type 'quit', 'exit', 'stop' to end our chat.


You:  get the list of tickets in jira epic sky-550


Agent: Here are the tickets in the epic SKY-550:

*   **SKY-564**: Enable asset actions in Marmind (Status: Selected for Development)
*   **SKY-563**: Assets usage tracking (Status: Backlog)
*   **SKY-562**: Enable asset metadata synchronisation (Status: In Progress)
*   **SKY-561**: Enable asset upload to DAM (Status: Backlog)
*   **SKY-560**: Implement user roles and permissions (Status: Done)
*   **SKY-558**: Implement asset assignment (Status: In Progress)
*   **SKY-557**: Enable asset preview in Marmind (Status: In Progress)
*   **SKY-556**: Enable asset search in Marmind (Status: In Progress)
*   **SKY-555**: Connect to the DAM system API (Status: In Progress)



You:  make analysis on the status transitions of the tickets in this epic


Agent: 🔍 [<function clear_status_transitions_data_set at 0x12b5b68e0>] State check: 0 transitions in context
🔍 [<function clear_status_transitions_data_set at 0x12b5b68e0>] Context ID: 4998684816
Status transitions set is empty!
🔍 [<function get_status_transitions_summary at 0x12b5b6480>] State check: 0 transitions in context
🔍 [<function get_status_transitions_summary at 0x12b5b6480>] Context ID: 4998681072
Current Status Transitions Data Summary
Transition Data: None - status transitions set is empty
Cached Changelog: None
I have cleared the previous data set. Now I will retrieve the changelog and extract status transitions for each ticket:

**SKY-564**: Enable asset actions in Marmind
**SKY-563**: Assets usage tracking
**SKY-562**: Enable asset metadata synchronisation
**SKY-561**: Enable asset upload to DAM
**SKY-560**: Implement user roles and permissions
**SKY-558**: Implement asset assignment
**SKY-557**: Enable asset preview in Marmind
**SKY-556**: Enable asset search in Marmin

...


📊 Status Time Bar Chart Created: Total Time Spent in Each Status for Epic SKY-550 Tickets
   • 2 active statuses analyzed
   • 21 total days across all statuses (includes self-loops)
   • 9 unique tickets
   • Excludes: Backlog, Closed, Done, New
   • Hover over bars to see ticket details

📈 Top statuses by time spent:
   1. In Progress: 12 days (7 entries)
   2. Selected for Development: 9 days (4 entries)

I have created a bar chart showing the total time spent in each status for the tickets in Epic SKY-550. This chart includes 2 statuses with a total of 21 days across all transitions.



You:  qiut


Agent: It seems like you want to quit. I'm here to help if you have any more JIRA ticket analysis or status flow visualization needs in the future. Goodbye!



You:  exit


👋 Goodbye!


In [6]:
# Remove all MCP auth tokens. This forces re-authentication next time
# !rm -rf ~/.mcp-auth

In [7]:
# Remove all persistent MCP/Atlassian data
# !rm -rf ~/.mcp* ~/.atlassian* ~/.config/mcp* ~/.config/atlassian* ~/.cache/mcp* ~/.cache/atlassian*